
$\newcommand{\br}{\mathbf{r}}$
$\newcommand{\brr}{\mathbf{r}}$
$\newcommand{\bs}{\mathbf{s}}$
$\newcommand{\bss}{\mathbf{s}}$

In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:80% !important; }</style>"))
display(HTML("<style>.container { width:80% !important; }</style>"))

# Metadynamics Hands-on

## Tricks from an old guys

Metadynamics is, at this moment, the most popular enhanced (ES) sampling technique out -there.

What is an ES technique?

\begin{equation}
    <O> = \frac{\int\ d\br \ O(\br)\ e^{-\beta U(\br)}}{\int\ d\br\ e^{-\beta U(\br)}}
\end{equation}
    
which in a MD or MC calculations is approximated by 
    
\begin{equation}
    <O> = \frac{1}{T}\int_0^T O(t) dt
\end{equation}


<center><img src='figures/high_fes.png'  style="width:40%"></center>


Let's help our simulation a little bit with regular Meta:
\begin{equation}
H(\br) = U(\br) + V(s,t) = U(\br) + h \sum_{t' = 0}^{T} g(s-s(t')) \\
\lim_{t \to \infty} F(s) \approx - V(s,t)
\end{equation}


better, with the Well-Tempered version!:
\begin{equation}
H(\br) = U(\br) + V(s,t) = U(\br) + h \sum_{t' = 0}^{T} g(s-s(t')) \exp{\left(- V(s,t')\frac{\gamma}{\gamma-1}\right)}\\
\lim_{t \to \infty} F(s) = - \frac{\gamma}{\gamma-1}V(s,t)
\end{equation}


<center><img src='figures/meta_working.png'  style="width:40%"></center>

The action of the external potential is to fill deep energy minimum, push you over boundaries (barriers) and explore new minima. At the end of the calculations the potential tends to flatten out, and you can recover the original FES.

sounds interesting! ...

What is **s** though?

###  CVs, what are they?

A CV **s** is a function of the microscopic coordinate of your system **s** =  f(**R**). 
Some simple examples are distances between atoms, angles such as those find in polypeptides.

\begin{equation}
d_{ab} = \sqrt{(\mathbf{R_a}-\mathbf{R_b})^2}
\end{equation}


Some more complex CVs are coordination number

\begin{equation}
c_a = \sum^N_b w(d_{ab}), \ \ w[0,1]
\end{equation}

$w$ is an activation function, similar to those used in Neural Network or in SOAP to switch the contribution. 

PLUMED-2 can leverage C++ objects oriented programming, and create arbitrary complex CVs. For example, the number of atom coordinated by 3 other atoms can be calculated as:

\begin{equation}
n_3 = \sum^N_b \exp{\left(-\frac{(c_a-3)^2}{\sigma^2}\right)}
\end{equation}


Choosing a CV is more art than science, but there are two things you should remember:

1. CVs should IDENTIFYING the metastable states that we are interested in 
2. CVs should ISOLATE the metastable states that we are interested in 



Usually, the CVs that are chosen are the slowest (i.e. those that have the slowest dynamics). It make sense to bias them, because it is harder to observe a transition in those CVs during a MD.

CVs that tends to work pretty well are distances, angles and dihedrals. This is becase there is a 1-to-1 correspondance is easy to achieve!

CVs that are powerful but harder to use and potentially problematic are those that are based or linear or non-linear combination of other CVs.

How do you set up a calculations?

### Easy with PLUMED-2.0

Plumed is an open-source software that can be patched to many MD code such as GROMACS, LAMMPS etc..
It is relatively easy to download it and install it, and there is a large community so you can easy find answer for the question you have

After the installation,  be sure that PLUMED can find the correct libraries with 
```bash
$ source /path/to/plumed/sourceme.sh
```

PLUMED then require a file with instruction on what to do during the calculation

```bash
# reduced units
UNITS NATURAL
# this line define the CV (s)
d1: DISTANCE ATOMS=1,2 COMPONENTS
# This lines are usually not present in a standard simulations. Here they represent the PES that we will sample
# since is a toy model
ff: MATHEVAL ARG=d1.x VAR=x0 PERIODIC=NO FUNC=x0^2*((1.05*x0-1)*(x0+1))*200
bb: BIASVALUE ARG=ff

# This define the metadynamics calculation
mt: METAD ...
ARG=d1.x
HEIGHT=0.5 SIGMA=0.05 BIASFACTOR=30
TEMP=1.0
GRID_MIN=-1.5 GRID_MAX=1.5 GRID_BIN=400
PACE=500
...

# print the value of the CV to a file, as well as the bias of the Metadynamics
PRINT ARG=d1.x,mt.bias FILE=colvar STRIDE=500
```

For the simple toy systems we will use, PLUMED will be the molecular dynamics engine AND the Metadynamics plugin. To run a MD we use the ```pesmd``` command
```bash
plumed pesmd < input
```
which contains the information about the MD calculation

```markdown
temperature 1
tstep 0.0005
friction 1
dimension 1 
nstep 4000000
ipos 0.75
periodic false
plumed plumed.dat

```

Ok, lets run:
1. An unbiased calculation to check if we are in the situation where we need to use ES
2. A biased calculation to recover the FES

**After the calculations please take a moment to plot:**
 1. The CV as a function of time (2nd col of HILLS of colvar file)
 2. The bias as a function of time (3rd col of colvar)
 3. The height of the gaussian repulsive functions (HILLS 4th col)
 4. How much bias you deposited along the CV (colvar col 3 vs col 2)

Once that the calculation is finished, reconstruct the final FES with 

```bash
plumed sum_hills --hills HILLS --mintozero --bin 200 --kt 1.0 --negbias
```

and check the profile of the FES

```bash
python ../../scripts/compare_results_reference.py --fes negativebias.dat
```

or 

```bash
python ../../scripts/compare_results_reference.py --fes negativebias.dat --ref reference/reference_fes.dat
```


### What about Free Energy Differences?

Free Energy Differences ($\Delta G$) are way more interesting to experimentalists than the FES. 
They are also good to estimate if our calculations converged or not, but for this we need to evaluate them as a function of time.

Calculating $\Delta G$ is easy with plumed, and a python script! First, create MANY fes, with a desired stride

```bash
plumed sum_hills --hills HILLS --mintozero --bin 200 --kt 1.0 --negbias --stride 100
```

This will generate many negativebias_XX.dat files. After this, use the script

```bash
python ../../scripts/calculate_fes_difference.py --prefix negativebias --mina -1 -0.5 --minb 0.5 1 --kt 1.0
```

In practice, this is done by selecting two parts of the phase space $a$ and $b$, and evaluating the probability rather than the FES. So using directly the PLUMED negativebias_XX.dat results:

\begin{equation}
P_a(t) = \int_{a-\delta}^{a+\delta}e^{-G(s,t)/kT} ds \\
P_b(t) = \int_{b-\delta}^{b+\delta}e^{-G(s,t)/kT} ds
\end{equation}
and then 

\begin{equation}
\Delta G_{ab}(t) = -kT \log \frac{P_a(t)}{P_b(t)}
\end{equation}


Using a single point as reference, is 
1. Incorrect, since there is a probability field, not "a structure"
2. Noisy, because as you decrease $\delta \to 0$ you get less and less statistics

### Thanks Fede, I will go do FES!

Or damages, most likely, because you are just grasping it atm.

<center><img src='figures/dunning-kruger.jpg'  style="width:50%"></center>


Let's redo everything with a slightly different change with **mono-dimensional-bad-behaving**.
**Mono-dimensional-bad-behaving**, has basically the same FES of **mono-dimensional**, but the difference in FES between $a$ and $b$ is slightly larger

Run the calculation, then check the FES and $\Delta G(t)$. Consider if the simulation is converged.

THEN

compare with the reference

### It's wrong! What is off!??


Yes of course you should simulate more, but how much?

Let's start by checking what the bias and the height of the gaussians as a function of the $s$ variable are doing! 

(colvar 3rd vs 2nd columns and HILLS 4th vs 2nd columns)

**A simulation is converged, once the height of the repulsive gaussian is 0, EVERYWHERE**

Sign that something is off,
**sometimes the convergence is drifting a bit.** Check the convergence as a function of time

### Ok fine I got it! Thanks


Yhea, but we have not finished yet.

Try the **bi-dimensiona-mono-cv**:

As you may have guessed, this system is a 2D systems but we are going to bias only 1 CV for the moment!

Calculate the FES $\Delta G(t)$, as well as the HILLS heights and bias as a function of $s$

THEN

Compare with the *proj_y* fes in the reference folder.

Surprise! The height behave in a good way, but he FES does not!!!!!
WHY?

because we  neglecting a **degrees of freedom**, that has an important role in the transition between the two minima!

There is a way for checking about neglected CVs, but I will discuss it later as last argument!
For now, lets do a **2D WT META** calculations

### Metadynamics in 2D

With 2D, the things get slightly more complex

Since we have two CVs, now the plumed file looks like

```markdown
mt: METAD ...
ARG=d1.x,d1.y
HEIGHT=1.0 SIGMA=0.05,0.05 BIASFACTOR=15
TEMP=1.0
GRID_MIN=-1.8,-1.8 GRID_MAX=1.8,1.8 GRID_BIN=400,400
PACE=500
...

PRINT ARG=d1.x,d1.y,mt.bias FILE=colvar STRIDE=500
```

notice that now we have two arguments, one per CV, two Sigmas, and two parameters per grid input parameter, since we now have a 2D grid

Run the calculation as you did for the 1D system, check all the important features (bias, heights, etc)

Of course, now you have two CVs, and to check the amount of bias deposited as a function of CVs you need to use a scatter plot with colorbar.

To recover the FES you need to explain to sum_hills that there is a 2D grid

```bash
 plumed sum_hills --hills HILLS --kt 1.0 --negbias --bin 200,200 --mintozero
 ```

If you want to evaluate the $\Delta G$ then you need to specify 2 extra boundaries per minimum, of course! You can do this with 

```bash
python ../../scripts/calculate_fes_difference.py --prefix negativebias --mina -1.5 0.5 -1.5 -0.5 --minb 0.5 1.5 -0.5 1.5 --kt 1.0
```

###  Can I recover  1D FES with sum_hills?

sure,  you can just integrate the extra dimension out
```bash
 plumed sum_hills --hills HILLS --kt 1.0 --negbias --bin 200,200 --mintozero --idw d1.x
 ```
 
 And the result will be only the fes along the CV named d1.x
 
 Check if the mono dimensional CV converged, and check the 2D FES as well.

It is worth saying that eventually, these calculations do converge, but they took a long time, because the evaluation of the FES is always off a bit.

This example is just to introduce you to the fact that there is ALWAYS an orthogonal CV. But in most cases is (sort-of) innocuous.

But they can be dangerous!

###  And to finish the worst case scenario!

What is worst to have a neglected CV?
To have a neglected CV with a twist!

Try first the **zeta-potential-mono**.

The idea beyond this case, is that you are convinced that your system can be described with 1 CVs and has 2 metastable states, just like the **mono-dimensional** case.

Run the calculations, analyze the trajectory by checking the height, bias as a function of time and of $s$

Calculate free energy differences, evaluate the FES and compare it with the one in the reference directoy which is *proj_y*

What do you think?

Now try to run the **zeta-potential** and check what is the problem!

This is a case where CV can identifying the minima, but not ISOLATE THEM.

###  Can we obtain information on other CVs or degrees of freedom?

It is possible to recover information on other CVs or degrees of freedom!
We can evaluate what would be the weight of each configuration sampled in our Metadynamics calculations by reweigthing, so to recover the Boltzmann weight that the coonfiguation would have in an unbiased MD run.

\begin{equation}
    \hat{P}(\br,t) = P(\br)\ e^{-\beta [V(\bs(\br),t) - c(t)]}, 
\end{equation}
in which 
\begin{equation}
 e^{-\beta c(t)} = \frac{\int\ d \bs\ P(\bs)\ e^{-\beta V(\bs,t)}}{\int d \bs\ P(\bs)},
\end{equation}

the unbiased probability can thus be obtained, starting from the biased one using
\begin{equation}
    \hat{P}(\br) = \int_0^T \ dt \ e^{\beta [V(\bs,t) - c(t)]} \delta(s-s(t)), 
\end{equation}

We can evaluate the renormalization constant c(t) by using

\begin{equation}
    e^{-\beta c(t)} = \frac{\int_0^t dt'\ e^{\beta[V(\bs(t'),t')-c(t')-V(\bs(t'),t)]}}{\int_0^t dt'\ e^{\beta [V(\bs(t'),t')-c(t')]}}.
\label{eq:time_c_t}
\end{equation}

A python module that implement this schema is available in cosmo-tools in python module named as ITRE
https://github.com/cosmo-epfl/cosmo-tools.git

Check the examples folder for more complex task. For now, we will reweight the 2D calculations that we have!

When you use ITRE there are a few things you need to know:
1. You can use a stride in the calculation so that you do not need to calculate c(t) every time Usually 10--50 is ok 
2. You need the instantaneous potential, and is better to use directly the one used from PLUMED. HOWEVER the ITRE recalculated potential and the PLUMED potential should be the same! CHECK THAT!
3. The weights that you calculate, and hence c(t), need to be aligned with your trajectory! so pay attention! They should start at the same time and have the same stride!
4. PLUMED does not exactly print the HEIGHT of the gaussin when you do Well Tempered, but rather an estimate of the FES. So you need to correct it! Is easy you can just normalize the whole heights and then multiply for the heights you set at the beginning.

Try to evaluate the FES in 2D using only 1 CV to bias in the  **reweight-** exercises in the exercises folder.



The *generate_ct.py* file use ITRE to evaluate c(t)
The *itre_reweight.py* file construct the reweighted FES and compare it with the analytic one.



As a matter of fact, the ITRE method is more precise in the evaluation of th FES than sum_hills and other reweigthing methods! 

The reason behind this is a little bit complex and buried in the thermodynamics approximation behind ITRE and sum_hillls and I decided to not discuss it yet!